In [39]:
import os
import gc
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm
from rasterstats import zonal_stats
from rasterio.io import MemoryFile

In [3]:
nuts_folder = r'C:\1-Data\NUTS'
for nuts_file in os.listdir(nuts_folder):
    if nuts_file.endswith('.shp'):
        print(nuts_file)

NUTS_RG_01M_2003_3035.shp
NUTS_RG_01M_2006_3035.shp
NUTS_RG_01M_2010_3035.shp
NUTS_RG_01M_2013_3035.shp
NUTS_RG_01M_2016_3035.shp
NUTS_RG_01M_2021_3035.shp


In [7]:
read_root = r'C:\1-Data\GHS_POP'
for folder in os.listdir(read_root):
    for pop_file in os.listdir(read_root + '\\' + folder):
        if pop_file.endswith('.tif'):
            print(pop_file)

GHS_POP_E1975_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E1980_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E1985_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E1990_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E1995_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E2000_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E2005_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E2010_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E2015_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E2025_GLOBE_R2023A_54009_100_V1_0.tif
GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0.tif


In [ ]:
zs_temp = pd.DataFrame(zonal_stats(nuts_folder + '\\' + nuts_file, read_root + '\\' + folder + '\\' + 'GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0.tif'))['mean']

In [ ]:
a = zonal_stats(nuts_folder + '\\' + nuts_file, read_root + '\\' + folder + '\\' + 'GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0.tif')

In [27]:
nuts = gpd.read_file(nuts_folder + '\\' + nuts_file)

In [33]:
EU = gpd.read_file(r'C:\1-Data\NUTS\EU.shp')

In [ ]:
rasterio.warp.reproject() 

In [53]:
def reproject_raster(in_path, dst_crs):
    with rasterio.open(in_path) as src:
        transform, width, height = rasterio.warp.calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

    with MemoryFile() as memfile:
        with memfile.open(**kwargs) as dst:
            for i in range(1, src.count + 1):
                rasterio.warp.reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=crs,
                    resampling=Resampling.nearest)
        with memfile.open() as dataset:  # Reopen as DatasetReader
            return dataset  # Note yield not return as we're a contextmanager

In [43]:
in_path = read_root + '\\' + folder + '\\' + 'GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0.tif'
dst_crs = 'EPSG:3035'
tif = reproject_raster(in_path, dst_crs)

In [55]:
re = reproject_raster(in_path, dst_crs) as re:
    print(re)

RasterioIOError: 087dcafe-aed6-46b2-bc15-058294a7d0a4.tif: Free disk space available is 33777467392 bytes, whereas 650362475568 are at least necessary. You can disable this check by defining the CHECK_DISK_FREE_SPACE configuration option to FALSE.